In [41]:
!apt-get install libmagic1
!pip install -U "pathway[all]"
!pip install openai==1.68.2
!pip install -U langgraph langchain-community langchainhub langchain-openai gradio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libmagic1 is already the newest version (1:5.41-3ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
  Using cached openai-1.60.2-py3-none-any.whl.metadata (27 kB)
  Using cached langchain-0.2.17-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_community-0.2.19-py3-none-any.whl.metadata (2.7 kB)
  Using cached langchain_core-0.2.43-py3-none-any.whl.metadata (6.2 kB)
  Using cached langchain_text_splitters-0.2.4-py3-none-any.whl.metadata (2.3 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
Using cached langchain-0.2.17-py3-none-any.whl (1.0 MB)
Using cached langchain_community-0.2.19-py3-none-any.whl (2.3 MB)
Using cached openai-1.60.2-py3-none-any.whl (456 kB)
Using cached langchain_core-0.2.43-py3-none-any.whl (397 kB)
Using cached langchain_text_splitters-0.2.4-py3-none-any.whl (25 kB)
Using cached aiofiles-24.1.0-py3-none-any.whl (1

In [ ]:
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
import time
import json
from typing import Iterable, Literal, List
from pydantic import BaseModel, Field
import pathway as pw
from langchain_community.vectorstores import PathwayVectorClient
from pathway.xpacks.llm.vector_store import VectorStoreServer, VectorStoreClient
from pathway.xpacks.llm import (
    embedders,
    llms,
    parsers,
    splitters,
)
from pathway.udfs import DiskCache
import warnings
warnings.filterwarnings("ignore")
# needed for the OpenAI embedder and the LLM we will use below, you can change the embedding provider, see the documentation:
# https://pathway.com/developers/api-docs/pathway-xpacks-llm/embedders
# days='1'
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

In [114]:
# folder that we will gather our docs in
DATA_PATH = "./data"

os.makedirs(DATA_PATH, exist_ok=True)

In [115]:
import re
from urllib.parse import urlparse
from langchain_community.document_loaders import WebBaseLoader


def load_page_content(url: str) -> str:
    """Load web page content with Langchain utilities."""
    return WebBaseLoader(url).load()[0].page_content


def ingest_webpage(url: str) -> None:
    """Save a webpage to local `DATA_PATH` folder."""
    text_content = load_page_content(url)

    parsed_url = urlparse(url)
    file_name = parsed_url.hostname + parsed_url.path.replace("/", "_") + ".txt"

    with open(os.path.join(DATA_PATH, file_name), "w", encoding="utf-8") as f:
        f.write(text_content)

def ingest_text_file(text_content: str, name: str) -> None:
    """Save a text file to local `DATA_PATH` folder."""
    # Using index to create unique and short filenames
    file_name = f"scraped_data_{name}.txt"

    with open(os.path.join(DATA_PATH, file_name), "w", encoding="utf-8") as f:
        f.write(text_content)

In [116]:
# ingest_webpage("https://www.espncricinfo.com/cricketers/virat-kohli-253802")

In [117]:
# host and port of the RAG app
pathway_host: str = "0.0.0.0"
pathway_port: int = 8000

In [118]:
# user_input = input("Ask something: ")


In [ ]:
import requests
import openai
import os

# Replace these with your credentials
GOOGLE_API_KEY = "Google_API_Key"

OPENAI_API_KEY = "Open_AI_KEY"

In [120]:
# Initialize OpenAI client
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  # Ensure API key is set as an environment variable

def query_constructor(query):
    system_prompt = '''You are an agent that specializes in sports knowledge. Modify the query to make it concise and effective for web search.
    The query should be modified in such a way that it give precise and accurate search result on google web search
    For example if the query is "what are the stats of virat kohli in t20"
    the response should be "Virat Kohli T20 stats latest"
    '''

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Make this query concise for web search: {query}"}
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )

    return response.choices[0].message.content + " -site:x.com -site:youtube.com -site:reddit.com -site:instagram.com -site:facebook.com -site:linkedin.com -site:thread.net "

# Example usage
# query = "virat kohli t20 stats"
# print(query_constructor(query))

In [ ]:
CX_ID = "CX_ID"

In [122]:
def google_search(query,search_days):
    """Fetch search results from Google Custom Search API"""
    url = f"https://www.googleapis.com/customsearch/v1?q={query}&key={GOOGLE_API_KEY}&cx={CX_ID}&dateRestrict=d{search_days}"
    print(url)
    response = requests.get(url)
    response.raise_for_status()
    results = response.json()
    print(results)

    snippets = []
    links=[]
    for item in results.get("items", []):
        snippets.append(item['snippet'])
        links.append(item['link'])
    return links,snippets  # Return top 10 search results


In [52]:
import pathway as pw
from pathway.stdlib.indexing import BruteForceKnnFactory, HybridIndexFactory
from pathway.stdlib.indexing.bm25 import TantivyBM25Factory
from pathway.udfs import DiskCache
from pathway.xpacks.llm import embedders, llms, parsers, splitters
from pathway.xpacks.llm.document_store import DocumentStore
from pathway.xpacks.llm.question_answering import BaseRAGQuestionAnswerer, RAGClient

In [53]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
import time
# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()


In [54]:



# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


# LLM with function call
llm = ChatOpenAI(model="gpt-4o", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question.
    If it is relevant to  the question to some extent also then also give yes
    give no in very very harsh scenario
    """

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader


In [55]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()



**AGENTIC RAG**

In [56]:
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )


# LLM with function call
llm = ChatOpenAI(model="gpt-4o", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeHallucinations)

# Prompt
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts or the text has some facts that can prove it.
     Dont be too harsh and give no in very very harsh scenario otherwise ouput yes in almost every scenario
     """

hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader
# hallucination_grader.invoke({"documents": docs, "generation": generation})

In [57]:
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )


# LLM with function call
llm = ChatOpenAI(model="gpt-4o", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeAnswer)

# Prompt
system = """You are a grader assessing whether an answer addresses / resolves a question \n
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader
# answer_grader.invoke({"question": question, "generation": generation})

In [58]:
#Function for transformaing the question into better prompt
llm = ChatOpenAI(model="gpt-4o", temperature=0)


def query_rewritter(question):
  # Prompt
  system = """You a question re-writer that converts an input question to a better version that is optimized \n
      for vectorstore retrieval. Look at the input and try to reason about the underlying semantic intent / meaning.
      For example if the query: What are the T20 statistics for Virat Kohli?
      Your response should be :  Virat Kohli t20 stats
      Use short words whenever possible like use  stats insted of statistics. Use HS instead of Highest Score and avg instead of Average

      Your response should be such that it is easy to retreive in the vector space. make it as short and crisp as possible
      """
  re_write_prompt = ChatPromptTemplate.from_messages(
      [
          ("system", system),
          (
              "human",
              "Here is the initial question: \n\n {question} \n Formulate an improved question.",
          ),
      ]
  )

  query_rewriter_agent = re_write_prompt | llm | StrOutputParser()
  return query_rewriter_agent.invoke({"question": question})

In [59]:
#For transformation of question into better question

llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Prompt
system = """You a question re-writer that converts an input question to a better version that is optimized \n
     for vectorstore retrieval. Look at the input and try to reason about the underlying semantic intent / meaning."""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
# question_rewriter.invoke({"question": question})

In [60]:
from typing import List

from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    input_days: str
    generation: str
    documents: List[str]
    commentary: int

In [61]:
def query_answer(query):
    system_prompt = '''
    You are an AI agent specializing in sports knowledge. Your task is to determine whether a given query can be answered based on your existing knowledge.

    Response Criteria:
    - Respond **"yes"** if the query pertains to **objective and factual sports knowledge** **within your knowledge cutoff (October 2023)**, such as:
      - Historical events, past tournaments, and iconic matches that occurred **on or before October 2023**.
      - Established rules, gameplay mechanics, and strategies.
      - Retired players, records, and achievements that **are permanently fixed**.

    - Respond **"no"** if the query:
      - Requires **real-time or frequently changing information**, including:
        - Live scores, current player statistics, recent rankings, or ongoing tournaments.
        - Any record held by an **active player (as of October 2023)**, since they can break their own records.
          - Example: *"What is the highest score of Rohit Sharma in IPL?" → "no"*
      - Is **opinion-based, subjective, or debatable**, such as:
        - *"Who is the greatest footballer of all time?"*
        - *"Which team has the best fan base?"*
      - Requires speculation, personal preference, or future predictions.

    Your response should be strictly **"yes"** or **"no"**—nothing else.

    '''

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"{query}"}
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )

    return response.choices[0].message.content

In [62]:
# Change the prompt


def query_type_answer(query):
    system_prompt = '''
    You are an AI agent specializing in sports knowledge. Your task is to determine whether a given query is asking for live commentary or not
    If it is asking for live commentary then respond with **"yes"** else **"no"**
    Even if it is asking for updates say no there should live commentary written in the prompt
    Your response should be strictly **"yes"** or **"no"**—nothing else.

    '''

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"{query}"}
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )

    return response.choices[0].message.content

In [63]:
def query_type_sport(query):
    system_prompt = '''
    You are an AI agent specializing in sports knowledge.Your task is to categorize about which sports is the query talking about
    You have to choose from ['football','cricket','basketball']
    Output just the name of the sport

    Your response should be strictly **"football"** or **"cricket"** or **"basketball"**—nothing else.
    '''

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"{query}"}
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )

    return response.choices[0].message.content

In [64]:
def commentary_agent(query):
  question=query['question']
  answer=query_type_sport(question)
  if answer=='football':
    links,snippets=google_search(query_constructor('site:https://www.sofascore.com/football/match/ '+ question),'1')
    link_choosen=commentary_link_assignment(question,links)
  elif answer=='cricket':
    links,snippets=google_search('site:www.cricbuzz.com/live-cricket-scores/ '+ question,'1')
    link_choosen=commentary_link_assignment(question,links)
  else:
    links,snippets=google_search(query_constructor('site:https://www.espn.com/ '+ question),'1')
    link_choosen=commentary_link_assignment(question,links)
  while True:
    print(link_choosen)
    live_commentary(link_choosen,3)
    generated = summary_agent('/content/data/diff.txt',question)
    return {"question": question, "generation": generated , "commentary":1}

In [65]:
# def commentary_agent(query):
#   question=query['question']
#   return {"question": question,"commentary":1}

In [66]:
def llm_agent(query):
    question=query['question']
    system_prompt = '''
    You are an AI agent specializing in sports knowledge. Your task is to provide the answer to the query
    '''

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"{question}"}
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )
    return {"question": question, "generation": response.choices[0].message.content}

In [67]:
def agent_decision(state):
  if query_type_answer(state["question"])=='yes':
    print("Going to commentary_agent")
    return "commentary_agent"
  else:
    if query_answer(state["question"])=='yes':
      print("Going to llm_agent")
      return "llm_agent"
    else:
      print("Going to data_creation_and_scarpping")
      return "data_creation_and_scarpping"

In [68]:
def data_creation_and_scarpping(state):
  question=state["question"]
  search_days=state["input_days"]
  links,snippets=google_search(query_constructor(question),str(search_days))
  print(google_search(query_constructor(question),str(search_days)))
  while len(links)==0:
    print("I am here")
    links,snippets=google_search(query_constructor(question),str(search_days))
  i=1
  for link in links[:7]:
    print(link)
    payload = { 'api_key': 'd80856c1689fbd8802e1dd90fd5f4e2c', 'url': f'{link}', 'device_type': 'desktop' }
    r = requests.get('https://api.scraperapi.com/', params=payload)
    # print(r.text)
    ingest_text_file(str(r.text),str(i))
    i+=1
  folder = pw.io.fs.read(
    path=f"{DATA_PATH}/*.txt",
    format="binary",
    with_metadata=True,

  )

  # list of data sources to be indexed
  sources = [folder]

  # define the document processing steps
  parser = parsers.UnstructuredParser()

  text_splitter = splitters.TokenCountSplitter(min_tokens=150, max_tokens=450)

  embedder = embedders.OpenAIEmbedder(cache_strategy=DiskCache())

  vector_server = VectorStoreServer(
      *sources,
      embedder=embedder,
      splitter=text_splitter,
      parser=parser,
  )

  t = vector_server.run_server(pathway_host, pathway_port, threaded=True)



def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    # pathway_client = VectorStoreClient(pathway_host, pathway_port)
    vectorstore_client = PathwayVectorClient(pathway_host, pathway_port)
    retriever = vectorstore_client.as_retriever()
    max_retries = 15  # Number of retries
    retry_delay = 5  # Seconds to wait before retrying

    for attempt in range(max_retries):
        try:
            docs = retriever.invoke({"question": query_rewritter(question)})  # No timeout param
            break  # Exit loop if successful
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            time.sleep(retry_delay)
    else:
        print("Failed after multiple attempts.")


    print(query_rewritter(question))
    return {"documents": docs, "question": question}


def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    print(question)
    print()
    # RAG generation
    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    rag_chain = prompt | llm | StrOutputParser()
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        # grade='yes'
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}


def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}


### Edges


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"


def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score.binary_score
    # Check hallucination
    # grade='yes'
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score.binary_score
        # grade='yes'
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

In [69]:

from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("commentary_agent",commentary_agent)
workflow.add_node("agent_decision",agent_decision)
workflow.add_node("llm_agent",llm_agent)
workflow.add_node("data_creation_and_scarpping",data_creation_and_scarpping)
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generate
workflow.add_node("transform_query", transform_query)  # transform_query

# Build graph
workflow.add_conditional_edges(
    START,
    agent_decision,
     {
         "llm_agent" : "llm_agent",
         "commentary_agent" : "commentary_agent",
         "data_creation_and_scarpping" : "data_creation_and_scarpping",
     },
)
workflow.add_edge("commentary_agent", END)
workflow.add_edge("llm_agent", END)
workflow.add_edge("data_creation_and_scarpping", "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",

        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
    },
)

# Compile
app = workflow.compile()

In [70]:
def commentary_link_assignment(question,links):
    system_prompt = f'''
    You are an AI agent specializing in sports knowledge. You will be given a list of links and a query.
    Your task is to output the most suitable link corresponding to that query
    just output the link and nothing else.

    Here are the links:
    {links}

   '''

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )

    return response.choices[0].message.content


In [71]:
# # For Internal Testing

# inputs = {
#     "question": "mi vs kkr Live Commentary 2025",
#     "input_days" : "1"
# }

In [72]:
# links,snippet=google_search(query_constructor('site:https://www.sofascore.com/football/match/ '+ "Torino vs Lazio Live Commentary latest"),'1')

In [73]:
# for link in links:
#   print(link)

In [74]:
# result = app.invoke(inputs)
# if result["commentary"]==1:

# result["generation"]

In [145]:
import tiktoken
from openai import OpenAI

client = OpenAI()

def summary_agent(txt_file, question, model="gpt-4o", batch_size=100000): # Add question as a parameter
    enc = tiktoken.encoding_for_model(model)

    # Read entire file
    with open(txt_file, "r", encoding="utf-8") as f:
        data = f.read()

    # Tokenize the whole document
    tokenized_data = enc.encode(data)

    # Split into batches of 100,000 tokens
    batches = [tokenized_data[i : i + batch_size] for i in range(0, len(tokenized_data), batch_size)]

    batch_summaries = []
    for i, batch in enumerate(batches):
        truncated_data = enc.decode(batch)  # Convert back to string

        system_prompt = f'''
        You are an AI agent specializing in sports knowledge. You will be given a txt file containing updates.
        Your task is to provide a summary of the updates

        Here is batch {i + 1}:
        {truncated_data}
        '''

        messages = [{"role": "system", "content": system_prompt}]

        response = client.chat.completions.create(
            model=model,
            messages=messages
        )

        batch_summaries.append(response.choices[0].message.content)

    # Combine batch summaries into a final summary
    combined_summary_prompt = f'''
    You have been given multiple batch summaries of a large document. Your task is to generate a concise Commentary of the updates on match referred in "{question}".
    You dont have to summarise the whole match just summarise the updates
    Here are the batch summaries:
    {batch_summaries}
    '''

    final_messages = [{"role": "system", "content": combined_summary_prompt}]

    final_response = client.chat.completions.create(
        model=model,
        messages=final_messages
    )

    return final_response.choices[0].message.content

In [146]:
import difflib
import requests
import time
def live_commentary(url,sleep_duration):

  payload = { 'api_key': 'd80856c1689fbd8802e1dd90fd5f4e2c', 'url': f'{url}', 'device_type': 'desktop' }
  r = requests.get('https://api.scraperapi.com/', params=payload)
  ingest_text_file(str(r.text),'old')
  time.sleep(sleep_duration)
  r = requests.get('https://api.scraperapi.com/', params=payload)
  ingest_text_file(str(r.text),'new')

  # Load the scraped data (assuming it's stored as text)
  with open("/content/data/scraped_data_old.txt", "r", encoding="utf-8") as f:
      old_data = f.readlines()

  with open("/content/data/scraped_data_new.txt", "r", encoding="utf-8") as f:
      new_data = f.readlines()

  # Compare the two versions
  diff = difflib.unified_diff(old_data, new_data, lineterm="")

  lines=[]
  # Print only the added lines
  for line in diff:
      if line.startswith("+") and not line.startswith("+++"):
          lines.append(line)
  with open("/content/data/diff.txt", "w") as file:
    file.write('\n'.join(lines))


In [ ]:
import gradio as gr
from langgraph.graph import StateGraph
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import time  # For typing effect

# Function to invoke LangGraph and maintain chat history
def app_invoke(question, chat_history, number_input):
    if chat_history is None:
        chat_history = []  # Initialize history

    # Add user message
    chat_history.append(("👤 You", question))

    # Simulating a typing effect
    time.sleep(0.5)

    if app is None:
        response = "Error: LangGraph app is not initialized."
    else:
        # Run the LangGraph workflow
        input_data = {"question": question, "history": chat_history, "input_days": number_input, "commentary":0}
        result = app.invoke(input_data)

        # Extract response
        response = result.get("generation", "No response generated.")

    # Add assistant response
    chat_history.append(("🤖 AI", response))

    return chat_history, chat_history  # Return updated state

# Define Gradio UI
with gr.Blocks(theme=gr.themes.Default(primary_hue="blue")) as interface:
    gr.HTML("<h1 style='text-align: center; color: #00aaff;'>⚽ AI-Powered Sports Chatbot</h1>")

    chatbot = gr.Chatbot(
        label="Chat History",
        bubble_full_width=False  # Ensures messages appear neatly
    )

    with gr.Row():
        number_input = gr.Number(label="Enter a number", interactive=True)
        user_input = gr.Textbox(
            label="Type your question...",
            placeholder="Ask me about any sport! 🏀⚽🎾",
            interactive=True
        )
        submit_btn = gr.Button("🚀 Send", variant="primary")

    with gr.Row():
        clear_btn = gr.Button("🗑️ Clear Chat", variant="stop")

    state = gr.State(value=[])  # Chat history storage

    # Define button click behavior
    def submit_question(user_text, number_value, chat_history):
        return app_invoke(user_text, chat_history, number_value)

    user_input.submit(submit_question, [user_input, number_input, state], [chatbot, state])
    submit_btn.click(submit_question, [user_input, number_input, state], [chatbot, state])

    def clear_chat():
        return [], []

    clear_btn.click(clear_chat, [], [chatbot, state])

# Launch Gradio UI
interface.launch(share=True)


In [ ]:
#Live Commentary

import gradio as gr
import time

# Generator function to simulate continuous commentary
def commentary_classify(user_input):
    question = user_input
    answer = query_type_sport(question)
    if answer == 'football':
        links, snippets = google_search(query_constructor('site:https://www.sofascore.com/football/match/ ' + question), '1')
        link_choosen = commentary_link_assignment(question, links)
    elif answer == 'cricket':
        links, snippets = google_search('site:www.cricbuzz.com/live-cricket-scores/ ' + question, '1')
        link_choosen = commentary_link_assignment(question, links)
    else:
        links, snippets = google_search(query_constructor('site:https://www.espn.com/ ' + question), '1')
        link_choosen = commentary_link_assignment(question, links)
    return link_choosen

# Generator function for continuous commentary in chat format (without user text)
def continuous_commentary(user_input):
    question = user_input
    chat_history = []
    if user_input:
        while True:
            live_commentary(commentary_classify(user_input), 3)
            generated = summary_agent('/content/data/diff.txt', question)
            chat_history.append((None, f'Bot: {generated}'))
            yield chat_history

# Gradio chat interface with continuous output
demo = gr.Interface(
    fn=continuous_commentary,
    inputs="text",
    outputs="chatbot",
    description="Enter some text and see continuous commentary in chat format.",
)

demo.launch()
